<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/DataWarehousing_with_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Datawarehousing with MySQL <br>

<br>
based on the codes available with the book Dimensional Datawarehousing with MySQL https://brainysoftware.com/9780975212820   [archive](https://drive.google.com/open?id=1BeIC5uEMGu85PADHRN-Zpn3lmZ3ihrkE) <br>
Updated to reflect MySQL V8

# Install MySQL

In [1]:
!apt update > /dev/null
!apt -y install mysql-server > /dev/null
!mysql --version
!/etc/init.d/mysql restart
#!mysql -e 'show databases'





mysql  Ver 8.0.32-0ubuntu0.20.04.2 for Linux on x86_64 ((Ubuntu))
 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


# Get Code Files<br>
 

In [2]:
#!wget https://brainysoftware.com/source/9780975212820.zip
#https://drive.google.com/open?id=1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0
#!gdown https://drive.google.com/uc?id=1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0
!wget -q -O dwCode.zip https://github.com/Praxis-QR/RDWH/blob/main/dwMySQL/dwCode.zip?raw=true
#!unzip 9780975212820.zip
!unzip dwCode.zip > /dev/null
!ls

dwCode.zip  sample_data  scripts  source_data


# Initial Setup

In [3]:
#Create userid
#
#!mysql < scripts/create_user_id.sql
#see the script that created the userid
#
#!cat scripts/create_user_id.sql
#
!mysql -e "create user dwid@localhost IDENTIFIED BY 'pw';"
!mysql -e "GRANT ALL PRIVILEGES ON *.* TO 'dwid'@'localhost'"

In [4]:
# see the privileges given to dw
#
#!mysql -e "show grants for dwid@localhost"

In [5]:
# Editing, updating the configuration file
!cat /etc/mysql/my.cnf

#
# The MySQL database server configuration file.
#
# You can copy this to one of:
# - "/etc/mysql/my.cnf" to set global options,
# - "~/.my.cnf" to set user-specific options.
# 
# One can use all long options that the program supports.
# Run program with --help to get a list of available options and with
# --print-defaults to see which it would actually understand and use.
#
# For explanations see
# http://dev.mysql.com/doc/mysql/en/server-system-variables.html

#
# * IMPORTANT: Additional settings that can override those from this file!
#   The files must end with '.cnf', otherwise they'll be ignored.
#

!includedir /etc/mysql/conf.d/
!includedir /etc/mysql/mysql.conf.d/


In [6]:
# saving password for ease of use
!echo '[client]' >> /etc/mysql/my.cnf
!echo 'user = "dwid"' >> /etc/mysql/my.cnf
!echo 'password = "pw"' >> /etc/mysql/my.cnf
# allowing data load
!echo '[mysqld]' >> /etc/mysql/my.cnf
!echo 'secure-file-priv = ""' >> /etc/mysql/my.cnf
# restart database
!/etc/init.d/mysql restart

 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [7]:
#Create databases
#
#!mysql -udwid -ppw < scripts/create_databases.sql
# see what the script had
#

!mysql -e "CREATE DATABASE dw"
!mysql -e "CREATE DATABASE source"
!mysql -e "show databases"

+--------------------+
| Database           |
+--------------------+
| dw                 |
| information_schema |
| mysql              |
| performance_schema |
| source             |
| sys                |
+--------------------+


In [8]:
#create the various tables
# one fact table and three dimension tables
#
#!mysql -udwid -ppw < scripts/create_dw_tables.sql
!mysql < scripts/create_dw_tables.sql
!mysql dw -e "show tables"

+------------------+
| Tables_in_dw     |
+------------------+
| customer_dim     |
| date_dim         |
| order_dim        |
| product_dim      |
| sales_order_fact |
+------------------+


In [9]:
#!mysql dw -e "desc customer_dim"
#!mysql dw -e "desc product_dim"
#!mysql dw -e "desc order_dim"

In [10]:
!date

Tue 31 Jan 2023 12:09:30 AM UTC


In [11]:
#this action cannot be performed except on a local machine 
!sudo date +%Y%m%d -s "20070201"

date: cannot set date: Operation not permitted
20070201


In [12]:
#!cat scripts/customer_sk.sql
#!mysql -udwid -ppw < scripts/customer_sk.sql
!mysql < scripts/customer_sk.sql
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2023-01-31     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2023-01-31     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2023-01-31     | 9999-12-31  |
+-------------+-------

In [13]:
#!mysql -udwid -ppw < scripts/more_customer_sk.sql
!mysql < scripts/more_customer_sk.sql
!mysql dw -e "select * from customer_dim"
# note we could not set the date to an earlier date by changing the computer time

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2023-01-31     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2023-01-31     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2023-01-31     | 9999-12-31  |
|           4 |       

In [14]:
#since it was not possible to set the system date on Colab, the date was changed manually
!mysql dw -e "update customer_dim set effective_date = '2007-02-01' "
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           4 |       

# Chap 2 - Slowly Changing Dimensions

**SCD1**


---







In [ ]:
!mysql dw --table -e 'drop table if exists customer_stg'

In [15]:
!cp source_data/customer2.csv /var/lib/mysql/dw/customer.csv
!ls /var/lib/mysql/dw

customer.csv	  date_dim.ibd	 product_dim.ibd
customer_dim.ibd  order_dim.ibd  sales_order_fact.ibd


In [16]:
#!mysql --local-infile=1 < scripts/create_customer_stg.sql
#!mysql --local-infile=1 -udwid -ppw < scripts/create_customer_stg.sql
!mysql < scripts/create_customer_stg.sql
!mysql dw -e "select * from customer_stg"

+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+
| customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state |
+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+
|               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             |
|               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             |
|               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             |
|               4 | Good Companies         | 9500 Scott St.          |             17050 | Mechanicsburg | PA             |
|               5 | Wonderful Shops        | 3333 Rossmoyne Rd.      |             17050 | Mechanicsburg | PA             |
|       

In [17]:
!cat scripts/scd1.sql

/*********************************************************************/
/*                                                                   */
/* scd1.sql                                                          */                  
/*                                                                   */
/*********************************************************************/

/* default database to dw                                            */

USE dw;

/* update existing customers                                         */

UPDATE customer_dim a, customer_stg b
SET a.customer_name = b.customer_name
WHERE     a.customer_number = b.customer_number
      AND a.expiry_date = '9999-12-31'
      AND a.customer_name <> b.customer_name
;

/* add new customers                                                 */

INSERT INTO customer_dim
SELECT
  NULL	
, customer_number
, customer_name
, customer_street_address
, customer_zip_code
, customer_city
, customer_state
, CURRENT_DATE
, '9999-12-31'

In [18]:
# One customer updated, one new customer added
!mysql  < scripts/scd1.sql
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 

In [19]:
#since it was not possible to set the system date on Colab, the date was changed manually
!mysql dw -e "update customer_dim set effective_date = '2007-02-02' where customer_number = 7 and customer_name = 'Distinguished Partners' "
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 

In [20]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name          | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Really Large Customers | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores           | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers       | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 

**SCD2**


---



In [21]:
!mysql dw -e "truncate product_dim"
!mysql dw -e "select * from product_dim"

In [22]:
!cat source_data/product2-1.txt

PRODUCT CODE,PRODUCT NAME,PRODUCT GROUP
1          Hard Disk                     Storage
2          Floppy Drive                  Storage

In [23]:
!cat source_data/product2-2.txt

PRODUCT CODE,PRODUCT NAME,PRODUCT GROUP
1          Hard Disk Drive               Storage                  
2          Floppy Drive                  Storage
3          LCD Panel                     Monitor

In [24]:
!mysql  < scripts/create_product_stg.sql
!mysql dw -e "desc product_stg"

+------------------+----------+------+-----+---------+-------+
| Field            | Type     | Null | Key | Default | Extra |
+------------------+----------+------+-----+---------+-------+
| product_code     | int      | YES  |     | NULL    |       |
| product_name     | char(30) | YES  |     | NULL    |       |
| product_category | char(30) | YES  |     | NULL    |       |
+------------------+----------+------+-----+---------+-------+


In [25]:
# this script needs a change
!cat scripts/load_product_stg.sql

/*********************************************************************/
/*                                                                   */
/* load_product_stg.sql                                              */                  
/*                                                                   */
/*********************************************************************/

/* default database to dw                                            */

USE dw;

/* clean up the staging table                                        */

TRUNCATE product_stg;

/* use LOAD DATA INFILE                                              */

LOAD DATA INFILE 'product.txt'
INTO TABLE product_stg
FIELDS TERMINATED BY ''
OPTIONALLY ENCLOSED BY ''
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES
( product_code
, product_name 
, product_category )
;

/* end of script                                                     */


In [26]:
#!cp scripts/load_product_stg.sql scripts/bak-load_product_stg.sql
#!cp scripts/bak-load_product_stg.sql scripts/load_product_stg.sql
#!sed -i "s/TERMINATED BY ''/TERMINATED BY '\\\t'/g" scripts/load_product_stg.sql
#!cat scripts/load_product_stg.sql

In [28]:
# backup file before changing
#!cp scripts/bak-load_product_stg.sql scripts/load_product_stg.sql
!cp scripts/load_product_stg.sql scripts/bak-load_product_stg.sql
#
# modify script for comma delimited data
#
!sed -i "s/TERMINATED BY ''/TERMINATED BY ','/g" scripts/load_product_stg.sql
!cat scripts/load_product_stg.sql

/*********************************************************************/
/*                                                                   */
/* load_product_stg.sql                                              */                  
/*                                                                   */
/*********************************************************************/

/* default database to dw                                            */

USE dw;

/* clean up the staging table                                        */

TRUNCATE product_stg;

/* use LOAD DATA INFILE                                              */

LOAD DATA INFILE 'product.txt'
INTO TABLE product_stg
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY ''
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES
( product_code
, product_name 
, product_category )
;

/* end of script                                                     */


In [29]:
#the datafile needs to be changed to insert , as delimiter
#
!cp source_data/product2-1.txt /var/lib/mysql/dw/product.txt
!sed -i "s/  /,/g" /var/lib/mysql/dw/product.txt  # replace double space with comma in file
!sed -i 's/,\{2,\}/,/g' /var/lib/mysql/dw/product.txt  # https://stackoverflow.com/questions/31400614/replace-multiple-commas-with-a-single-one-linux-command
!sed -i "s/, /,/g" /var/lib/mysql/dw/product.txt  # remove space after comma
!cat /var/lib/mysql/dw/product.txt

PRODUCT CODE,PRODUCT NAME,PRODUCT GROUP
1,Hard Disk,Storage
2,Floppy Drive,Storage

In [30]:
!mysql < scripts/load_product_stg.sql
#!mysql -udwid -ppw --table < pm_load_prod_stg.sql
!mysql dw -e "select * from product_stg"

+--------------+--------------+------------------+
| product_code | product_name | product_category |
+--------------+--------------+------------------+
|            1 | Hard Disk    | Storage          |
|            2 | Floppy Drive | Storage          |
+--------------+--------------+------------------+


In [31]:
!cat scripts/scd2.sql

/*********************************************************************/
/*                                                                   */
/* scd2.sql                                                          */                  
/*                                                                   */
/*********************************************************************/

/* default database to dw                                            */
                 

USE dw;

/* expire the existing product					 */

UPDATE 
  product_dim a
, product_stg b
SET
  expiry_date = SUBDATE(CURRENT_DATE, 1)
WHERE
    a.product_code = b.product_code
AND (   a.product_name <> b.product_name
     OR a.product_category <> b.product_category )
AND expiry_date = '9999-12-31'
;

/* add a new row for the changing product                           */
                   

INSERT INTO product_dim 
SELECT
  NULL	
, b.product_code
, b.product_name
, b.product_category
, CURRENT_DATE
, '9999-12-31'
FROM 
  pro

In [32]:
!mysql < scripts/scd2.sql
!mysql dw -e "select * from product_dim"

+------------+--------------+--------------+------------------+----------------+-------------+
| product_sk | product_code | product_name | product_category | effective_date | expiry_date |
+------------+--------------+--------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk    | Storage          | 2023-01-31     | 9999-12-31  |
|          2 |            2 | Floppy Drive | Storage          | 2023-01-31     | 9999-12-31  |
+------------+--------------+--------------+------------------+----------------+-------------+


In [33]:
!mysql dw -e "update product_dim set effective_date = '2007-02-03'"
!mysql dw -e "select * from product_dim"

+------------+--------------+--------------+------------------+----------------+-------------+
| product_sk | product_code | product_name | product_category | effective_date | expiry_date |
+------------+--------------+--------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk    | Storage          | 2007-02-03     | 9999-12-31  |
|          2 |            2 | Floppy Drive | Storage          | 2007-02-03     | 9999-12-31  |
+------------+--------------+--------------+------------------+----------------+-------------+


In [34]:
# new product file
#
!cp source_data/product2-2.txt /var/lib/mysql/dw/product.txt
!sed -i "s/  /,/g" /var/lib/mysql/dw/product.txt  # replace space with comma in file
!sed -i 's/,\{2,\}/,/g' /var/lib/mysql/dw/product.txt  # https://stackoverflow.com/questions/31400614/replace-multiple-commas-with-a-single-one-linux-command
!sed -i "s/, /,/g" /var/lib/mysql/dw/product.txt  # remove space after comma
!cat /var/lib/mysql/dw/product.txt

PRODUCT CODE,PRODUCT NAME,PRODUCT GROUP
1,Hard Disk Drive,Storage,
2,Floppy Drive,Storage
3,LCD Panel,Monitor

In [35]:
# One updated, one new
!mysql < scripts/load_product_stg.sql
#!mysql -udwid -ppw --table < pm_load_prod_stg_02.sql
!mysql dw -e "select * from product_stg"

+--------------+-----------------+------------------+
| product_code | product_name    | product_category |
+--------------+-----------------+------------------+
|            1 | Hard Disk Drive | Storage          |
|            2 | Floppy Drive    | Storage          |
|            3 | LCD Panel       | Monitor          |
+--------------+-----------------+------------------+


In [36]:
# One updated, one new
!mysql  < scripts/scd2.sql
!mysql dw -e "select * from product_dim"

+------------+--------------+-----------------+------------------+----------------+-------------+
| product_sk | product_code | product_name    | product_category | effective_date | expiry_date |
+------------+--------------+-----------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk       | Storage          | 2007-02-03     | 2023-01-30  |
|          2 |            2 | Floppy Drive    | Storage          | 2007-02-03     | 9999-12-31  |
|          4 |            1 | Hard Disk Drive | Storage          | 2023-01-31     | 9999-12-31  |
|          5 |            3 | LCD Panel       | Monitor          | 2023-01-31     | 9999-12-31  |
+------------+--------------+-----------------+------------------+----------------+-------------+


In [37]:
!mysql dw -e "update product_dim set effective_date = '2007-02-05' where effective_date = '2023-01-28'"
!mysql dw -e "update product_dim set expiry_date = '2007-02-04' where expiry_date = '2023-01-27'"
#there seems to be an inexplicable difference between the output shown here and the output shown in the book
#product_sk should be 1,2,3,4 and not 1,2,4,5 
#it seems autoincrements behave erratically and hence should not be used as surrogate keys as shown here
!mysql dw -e "select * from product_dim"

+------------+--------------+-----------------+------------------+----------------+-------------+
| product_sk | product_code | product_name    | product_category | effective_date | expiry_date |
+------------+--------------+-----------------+------------------+----------------+-------------+
|          1 |            1 | Hard Disk       | Storage          | 2007-02-03     | 2023-01-30  |
|          2 |            2 | Floppy Drive    | Storage          | 2007-02-03     | 9999-12-31  |
|          4 |            1 | Hard Disk Drive | Storage          | 2023-01-31     | 9999-12-31  |
|          5 |            3 | LCD Panel       | Monitor          | 2023-01-31     | 9999-12-31  |
+------------+--------------+-----------------+------------------+----------------+-------------+


#Chap 3 -  Measure Additivity

In [38]:
!cat scripts/additive_data.sql

/*********************************************************************/
/*                                                                   */
/* additive_data.sql                                                 */                  
/*                                                                   */
/*********************************************************************/

USE dw;

INSERT INTO order_dim VALUES
  (NULL, 1, CURRENT_DATE, '9999-12-31')
, (NULL, 2, CURRENT_DATE, '9999-12-31')
, (NULL, 3, CURRENT_DATE, '9999-12-31')
, (NULL, 4, CURRENT_DATE, '9999-12-31')
, (NULL, 5, CURRENT_DATE, '9999-12-31')
, (NULL, 6, CURRENT_DATE, '9999-12-31')
, (NULL, 7, CURRENT_DATE, '9999-12-31')
, (NULL, 8, CURRENT_DATE, '9999-12-31')
, (NULL, 9, CURRENT_DATE, '9999-12-31')
, (NULL, 10, CURRENT_DATE, '9999-12-31')
;

INSERT INTO date_dim VALUES
  (NULL, '2005-10-31', 'October', 10, 4, 2005, CURRENT_DATE, '9999-12-31')
;

INSERT INTO sales_order_fact VALUES
  (1, 1, 2, 1, 1000)
, (2, 2, 3, 1, 1

In [39]:
!mysql < scripts/additive_data.sql
!mysql dw -e "select * from sales_order_fact"

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
+----------+-------------+------------+---------------+--------------+


In [40]:
!mysql --table < scripts/across_all_dimensions.sql

+---------------------+
| sum_of_order_amount |
+---------------------+
|            58000.00 |
+---------------------+


In [41]:
!mysql --table < scripts/across_date_product.sql
# note this adds up to 58000 as well, as it should

+-----------------+---------------------+
| customer_number | sum_of_order_amount |
+-----------------+---------------------+
|               1 |             7000.00 |
|               2 |             9000.00 |
|               3 |            12000.00 |
|               4 |            14000.00 |
|               5 |            16000.00 |
+-----------------+---------------------+


In [42]:
%%writefile pm_across_date_customer_order.sql
USE dw;
SELECT
product_code, SUM(order_amount) sum_of_order_amount
FROM
sales_order_fact a
, product_dim b
WHERE
a.product_sk = b.product_sk
GROUP BY
product_code
;
/* end of script */

Writing pm_across_date_customer_order.sql


In [43]:
!mysql --table < pm_across_date_customer_order.sql
# this does not add up to 58,000 ! error !! 

+--------------+---------------------+
| product_code | sum_of_order_amount |
+--------------+---------------------+
|            2 |            23000.00 |
|            1 |            20000.00 |
+--------------+---------------------+


In [44]:
#trying to fix the error referred to above
!mysql dw -e "update product_dim set product_sk = 3 where product_sk = 4"
!mysql dw -e "update product_dim set product_sk = 4 where product_sk = 5"


In [45]:
!mysql -e 'USE dw; \
           SELECT * from sales_order_fact; \
           select * from product_dim;'
#!mysql -udwid -ppw --table < pm_quickcheck.sql

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
+----------+-------------+------------+---------------+--------------+
+-----

In [46]:
!mysql --table < scripts/across_date_customer.sql
# now we get back 58,000

+--------------+---------------------+
| product_code | sum_of_order_amount |
+--------------+---------------------+
|            2 |            23000.00 |
|            1 |            15000.00 |
|            3 |            20000.00 |
+--------------+---------------------+


In [ ]:
#after the error fix, back in sync with the tutorial book

In [47]:
!mysql --table < scripts/across_date.sql
# and we are back to 58,000 again

+-----------------+--------------+---------------------+
| customer_number | product_code | sum_of_order_amount |
+-----------------+--------------+---------------------+
|               1 |            2 |             1000.00 |
|               2 |            1 |             1000.00 |
|               3 |            3 |             4000.00 |
|               4 |            2 |             4000.00 |
|               5 |            1 |             6000.00 |
|               1 |            3 |             6000.00 |
|               2 |            2 |             8000.00 |
|               3 |            1 |             8000.00 |
|               4 |            3 |            10000.00 |
|               5 |            2 |            10000.00 |
+-----------------+--------------+---------------------+


#Chap 4 - Dimensional Queries

In [48]:
#!ls scripts/da*

In [49]:
!cat scripts/dimensional_query_data.sql 

/*********************************************************************/
/*                                                                   */
/* dimensional_query_data.sql                                        */                  
/*                                                                   */
/*********************************************************************/

USE dw;

INSERT INTO order_dim VALUES
  (NULL, 11, CURRENT_DATE, '9999-12-31')
, (NULL, 12, CURRENT_DATE, '9999-12-31')
, (NULL, 13, CURRENT_DATE, '9999-12-31')
, (NULL, 14, CURRENT_DATE, '9999-12-31')
, (NULL, 15, CURRENT_DATE, '9999-12-31')
, (NULL, 16, CURRENT_DATE, '9999-12-31')
;

INSERT INTO date_dim VALUES
  (NULL, '2005-11-01', 'November', 11, 4, 2005, CURRENT_DATE, '9999-12-31')
;

INSERT INTO sales_order_fact VALUES
  (11, 1, 2, 2, 20000) 
, (12, 2, 3, 2, 25000)
, (13, 3, 4, 2, 30000)
, (14, 4, 2, 2, 35000) 
, (15, 5, 3, 2, 40000)
, (16, 1, 4, 2, 45000)
;

/* end of script                                

In [50]:
!mysql dw --table < scripts/dimensional_query_data.sql
!mysql dw -e "select * from sales_order_fact"

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
|       11 |           1 |          2 |             2 |     20000.00 |
|     

In [51]:
!cat scripts/daily_aggregation.sql

/*********************************************************************/
/*                                                                   */
/* daily_aggregation.sql                                             */                  
/*                                                                   */
/*********************************************************************/

SELECT date, SUM(order_amount), COUNT(*)
FROM
  sales_order_fact a
, date_dim b
WHERE
a.order_date_sk = b.date_sk
GROUP BY date
ORDER BY date
;

/* end of script                                                     */


In [52]:
!mysql dw --table < scripts/daily_aggregation.sql

+------------+-------------------+----------+
| date       | SUM(order_amount) | COUNT(*) |
+------------+-------------------+----------+
| 2005-10-31 |          58000.00 |       10 |
| 2005-11-01 |         195000.00 |        6 |
+------------+-------------------+----------+


In [53]:
!mysql dw --table < scripts/annual_aggregation.sql

+------+-----------------+---------------+-------------------+----------+
| year | product_name    | customer_city | SUM(order_amount) | COUNT(*) |
+------+-----------------+---------------+-------------------+----------+
| 2005 | Floppy Drive    | Mechanicsburg |          70000.00 |        5 |
| 2005 | Floppy Drive    | Pittsburgh    |           8000.00 |        1 |
| 2005 | Hard Disk Drive | Mechanicsburg |          46000.00 |        2 |
| 2005 | Hard Disk Drive | Pittsburgh    |          34000.00 |        3 |
| 2005 | LCD Panel       | Mechanicsburg |          61000.00 |        3 |
| 2005 | LCD Panel       | Pittsburgh    |          34000.00 |        2 |
+------+-----------------+---------------+-------------------+----------+


In [54]:
!mysql dw --table < scripts/monthly_storage.sql

+-----------------+------------+------+-------------------+----------+
| product_name    | month_name | year | SUM(order_amount) | COUNT(*) |
+-----------------+------------+------+-------------------+----------+
| Floppy Drive    | November   | 2005 |          55000.00 |        2 |
| Hard Disk Drive | November   | 2005 |          65000.00 |        2 |
| Floppy Drive    | October    | 2005 |          23000.00 |        4 |
| Hard Disk Drive | October    | 2005 |          15000.00 |        3 |
+-----------------+------------+------+-------------------+----------+


In [55]:
!mysql dw --table < scripts/quarterly_mechanicsburg.sql

+---------------+---------+------+-------------------+-----------------+
| customer_city | quarter | year | SUM(order_amount) | COUNT(order_sk) |
+---------------+---------+------+-------------------+-----------------+
| Mechanicsburg |       4 | 2005 |         177000.00 |              10 |
+---------------+---------+------+-------------------+-----------------+


In [56]:
!cat scripts/monthly_product_performer.sql
# there is an error in this script .. can you spot it?

/*********************************************************************/
/*                                                                   */
/* monthly_product_performer.sql                                     */                  
/*                                                                   */
/*********************************************************************/

USE dw;

SELECT month_name, year, product_name, SUM(order_amount), COUNT(*)
FROM
  sales_order_fact a
, product_dim b
, date_dim c
WHERE
    a.product_sk = b.product_sk
AND a.order_date_sk = c.date_sk
GROUP BY 
  month_name
, year
, product_name
HAVING SUM(order_amount) >= 75000
ORDER BY 
  month
, year
, product_name
;

/* end of script                                                     */


In [57]:
!mysql dw --table < scripts/monthly_product_performer.sql

ERROR 1055 (42000) at line 9: Expression #1 of ORDER BY clause is not in GROUP BY clause and contains nonaggregated column 'dw.c.month' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by


In [58]:
%%writefile pm_monthly_product_performer.sql

USE dw;

SELECT month_name, year, product_name, SUM(order_amount), COUNT(*)
FROM
  sales_order_fact a
, product_dim b
, date_dim c
WHERE
    a.product_sk = b.product_sk
AND a.order_date_sk = c.date_sk
GROUP BY 
  month_name
, year
, product_name
HAVING SUM(order_amount) >= 75000
ORDER BY 
  month_name
, year
, product_name
;

Writing pm_monthly_product_performer.sql


In [59]:
#running the corrected script
!mysql dw --table < pm_monthly_product_performer.sql

+------------+------+--------------+-------------------+----------+
| month_name | year | product_name | SUM(order_amount) | COUNT(*) |
+------------+------+--------------+-------------------+----------+
| November   | 2005 | LCD Panel    |          75000.00 |        2 |
+------------+------+--------------+-------------------+----------+


sample forms https://colab.research.google.com/notebooks/forms.ipynb

In [60]:
!mysql dw --table < scripts/loyal_customer.sql

+-----------------+------+----------+
| customer_number | year | COUNT(*) |
+-----------------+------+----------+
|               1 | 2005 |        4 |
+-----------------+------+----------+


In [61]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Singapore')))

signed off at   2023-01-31 08:20:16.203791+08:00


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)